# Notebook Overview


  <div align="center" style="margin: 25px 0; padding: 15px; background-color: white; border: 1px solid #444; border-radius: 8px;">
    <img src="./images/architecture.jpg" alt="vNetRunner Overview" width="300"" />
  </div>




- Data Loading
- Data Preprocessing
    - Deleting columns with no variation
    - Data normalization
    - Removing the slient datapoints from other other slices
    - Data Visualization

- Model Training
- Exercise: Window Size

## Importing the necessary libraries

In [ ]:
import os
import time
import pickle
import torch
import pandas as pd
import numpy as np
from glob import glob
from torch import nn
import matplotlib.pyplot as plt
from scipy.stats import norm as normal
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import confusion_matrix as conf_mat
from sklearn.manifold import TSNE

pd.set_option('display.max_columns', None)

## Investigating the Files

In [ ]:
embb_csv = "./logs/SingleUE/Raw/embb*.csv"
urrl_csv = "./logs/SingleUE/Raw/urll*.csv"
mmtc_csv = "./logs/SingleUE/Raw/mmtc*.csv"
clean_csv = "./logs/SingleUE/Raw/null*.csv"

embb_files = glob(embb_csv)
embb_logs = [pd.read_csv(f, sep=",").dropna(how='all', axis='columns') for f in embb_files]

mmtc_files = glob(mmtc_csv)
mmtc_logs = [pd.read_csv(f, sep=",").dropna(how='all', axis='columns') for f in mmtc_files]

urll_files = glob(urrl_csv)
urll_logs = [pd.read_csv(f, sep=",").dropna(how='all', axis='columns') for f in urll_files]

ctrl_files = glob(clean_csv)
ctrl_logs = [pd.read_csv(f, sep=",").dropna(how='all', axis='columns') for f in ctrl_files]

<div style="font-family: Arial, sans-serif; color: #222; line-height: 1.6; max-width: 700px; margin: 20px auto; padding: 20px; border: 1px solid #444; border-radius: 8px; background-color: #eaeaea; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.15);">
  <h2 style="color: #ffffff; background-color: #333333; padding: 10px; border-radius: 4px; margin-bottom: 20px; text-align: center;">Exercise: Explore the Dataset</h2>
  
  <p style="font-size: 1.1em; margin-top: 15px; margin-bottom: 20px;">
    Take a few minutes to examine the dataset. Look at the data structure, features, and any patterns that stand out. Once you are familiar with the dataset, try answering the following questions. To get familiar, we have included a the cell above. Uncomment suitable lines in the cell below to see the output. 
  </p>
  
  <h3 style="color: #ff9800; margin-bottom: 15px;">Questions to Explore</h3>
  <ol style="font-size: 1.1em; margin-left: 25px; margin-bottom: 20px;">
    <li>What do any of the variable classes represent? What are EMBB, MMTC and URLLC logs here?</li>
    <li>What are the features of the each of the csv files we loaded?</li>
    <li>These data are collected from an xApp. What was the sampling granularity of our data? <b>Hint:</b> look at the <code>Timestamp</code> column? (Hint: Uncomment the last line in the cell below)</li> 
  </ol>
</div>


<center></center>

<!-- 
Q1. For every type of traffic, we have collected a set of traces. For instace, for EMBB traces, we have `embb_logs` list where each item contains a tabular dataset for that EMBB trace. 

Q2. Each trace loaded is a python dataframe including 31 columns that are metrics of the UE collected every 250ms. We can see values like `dl_mcs`, `dl_buffer` size and so on. The min, max and standard deviation for values of every columns are also specified.

Q3. As we have stated, data is collected every 250ms. So the data sampling granularity is 250ms. Now this data is sent from the RIC to the RAN not necesarily every 250ms, but possibly every few seconds. So even though the data is sampled every 250ms it is aggregated and sent from the RAN to RIC at lower granularities.

 -->

In [ ]:
# uncomment only one of the following lines to inspect one of the loaded datasets 
# embb_logs[0].describe()
# mmtc_logs[0].describe()
# urll_logs[0].describe()
# ctrl_logs[0].describe()


embb_logs[0].head()

## Data Preprocessing

### Removing some columns and Stacking the dataset


By looking at only one row of data, we might not possibly say what the original application producing this datapoint was. 
Was it an eMBB application? or was it a URLLC application. But if we look at some continuous rows, we can have more clues about the original application. For this reason, we use the variable `window_size` to stack rows of the original dataset to form new datapoints. Each new datapoint will have information about `window_size` consecutive rows.

In [ ]:
# number of previous rows to stack
window_size = 4

We will also remove some columns that are irrelevant to traffic classification task.
Furthermore, we aggregate all different traces of all different traffic classes into a single variable `data_in`. 
Plus, we store the class (label) of each datapoint in `data_lbl` variable.

In [ ]:
# In the this cell we drop some of the columns from each of the dataframes
# Then, we stack window_size rows so each datapoint itself, is represents a consecutive window_size datapoints of the original dataset
# We then have concatenated the dataframes of different types into one array with their corresponding labels in
# `trails_in` and `trials_lbl` variables


embb_traces = []
embb_labels = []
mmtc_traces = []
mmtc_labels = []
urll_traces = []
urll_labels = []
ctrl_traces = []
ctrl_labels = []


columns_drop = "Timestamp num_ues IMSI RNTI slicing_enabled slice_id slice_prb scheduling_policy".split(" ")
ctrl_class = 3

# stacks `window_siz`e number of rows on top of each other to form a dataseries
def make_windows(ds, window_size):
    new_ds = []
    for i in range(ds.shape[0]):
        if i + window_size < ds.shape[0]:
            new_ds.append(
                ds[i:i + window_size]  # slice
            )
    new_ds = np.array(new_ds)
    return new_ds


def check_slices(data, index, check_zeros=False):
    labels = np.ones((data.shape[0],), dtype=np.int32)*index
    if not check_zeros:
        return labels
    for i in range(data.shape[0]):
        sl = data[i]
        zeros = (sl == 0).astype(int).sum(axis=1)
        if (zeros > 10).all():
            labels[i] = 3 # control if all KPIs rows have > 10 zeros
    return labels


for ix, ds in enumerate([embb_logs, mmtc_logs, urll_logs, ctrl_logs]):
    for trace in ds:
        all_cols_names = trace.columns.values

        # We drop some of the columns here 
        columns_dropped = trace.drop(columns_drop, axis=1, inplace=False)
        cols_names = columns_dropped.columns.values                        # after drop
        new_trace = make_windows(columns_dropped, window_size)
            
        if ix == 0: # eMBB class
            embb_traces.append(new_trace)
            embb_labels.append(check_slices(new_trace, ix, False))         # labels are numbers (i.e. no 1 hot encoded)
        elif ix == 1: # MMTC class
            mmtc_traces.append(new_trace)
            mmtc_labels.append(check_slices(new_trace, ix, False))
        elif ix == 2: # URLLC class
            urll_traces.append(new_trace)
            urll_labels.append(check_slices(new_trace, ix, False))
        elif ix == 3:
            ctrl_traces.append(new_trace)
            ctrl_labels.append(np.ones((new_trace.shape[0],), dtype=np.int32) * ix)

data_in = np.concatenate((np.concatenate(embb_traces),
                            np.concatenate(mmtc_traces),
                            np.concatenate(urll_traces),
                            np.concatenate(ctrl_traces)), axis=0).astype(np.float32)
data_lbl = np.concatenate((np.concatenate(embb_labels),
                             np.concatenate(mmtc_labels),
                             np.concatenate(urll_labels),
                             np.concatenate(ctrl_labels)), axis=0).astype(int)


# Question: What is the dimension of the `data_in` variable? 
# What does the output of this cell show?



print(f"dataset shape: {data_in.shape}, labels shape: {data_lbl.shape}")

print("Number of datapoints:", data_in.shape[0])
print("size of each datapoint:", data_in.shape[1:])



### Normalize the values of each column to a range of [0, 1]

In [ ]:
# Here we look at all input features, and normalize the corresponding values to a range of [0, 1]
# This enhances the performance of the machine learning models as they work better when the range of 
# input values is in [0, 1]


def extract_feats_stats(cols_names, din):
    columns_maxmin = {}
    for c in range(din.shape[2]):
        col_max = din[:, :, c].max()
        col_min = din[:, :, c].min()
        col_mean = din[:, :, c].mean()
        col_std = din[:, :, c].std()
        columns_maxmin[c] = {'max': col_max, 'min': col_min, 'mean': col_mean, 'std': col_std, 'name': cols_names[c]}
    return columns_maxmin


def normalize_KPIs(columns_maxmin, din, doPrint=False):
    trials_in_norm = din.copy()
    for c, max_min_info in columns_maxmin.items():
        if isinstance(c, int):
            if max_min_info['name'] != 'Timestamp':
                col_max = max_min_info['max']
                col_min = max_min_info['min']
                if not (col_max == col_min):
                    if doPrint:
                        print('Normalizing Col.', max_min_info['name'], ' -- Max', col_max, ', Min', col_min)
                    trials_in_norm[:, :, c] = (trials_in_norm[:, :, c] - col_min) / (col_max - col_min)
                else:
                    trials_in_norm[:, :, c] = 0  # set all data as zero (we don't need this info cause it never changes)
            else:
                if doPrint:
                    print('Skipping normalization of Col. ', max_min_info['name'])

    return trials_in_norm

columns_maxmin = extract_feats_stats(cols_names, data_in)
data_in_norm = normalize_KPIs(columns_maxmin, data_in, doPrint=True)

print(f"X shape: {data_in.shape}, Y shape: {data_lbl.shape}")

### Remove columns with no variation

In [ ]:
no_variation_col_idx = []
no_variation_col_name = []
for k, v in columns_maxmin.items():
    if v['std'] == 0.0:


        print(f"feature {v['name']} removed because of its std is 0" )
        no_variation_col_idx.append(k)
        no_variation_col_name.append(v['name'])

trials_in = data_in.copy()
trials_in = np.delete(trials_in, no_variation_col_idx, axis=-1)
trials_in_norm = np.delete(data_in_norm, no_variation_col_idx, axis=-1)
new_cols_names = np.delete(cols_names, no_variation_col_idx)
columns_maxmin = extract_feats_stats(new_cols_names, trials_in)

print(f"New X shape: {trials_in.shape}, Y shape: {data_lbl.shape}")

### Shuffle the dataset

One of the common steps in ml data processing is to divide the data into two splits. The training dataset and the evaluation dataset. The training dataset is used for training the ml model and the evaluation dataset is used for evluating the trained model so that model is evaluated on datapoints it has not seen. 

By shuffling the data, we make sure a different subset of points fall into the training and test splits every time we run the notebook from the top to bottom.

In [ ]:
samp_ixs = list(range(trials_in.shape[0]))
np.random.shuffle(samp_ixs)                       # permutation
trials_in = trials_in[samp_ixs, :, :]
trials_in_norm = trials_in_norm[samp_ixs, :, :]
trials_lbl = data_lbl[samp_ixs]

### Spliting the dataset

In [ ]:
train_validation_ratio = 0.8

n_samps = trials_in.shape[0]
n_train = int(n_samps * train_validation_ratio)
n_valid = n_samps - n_train
nclasses = 3

trials_ds = {
    'train': {
        'samples': {
            'norm': torch.Tensor(trials_in_norm[0:n_train])
        },
        'labels': torch.Tensor(trials_lbl[0:n_train]).type(torch.LongTensor)
    },
    'valid': {
        'samples': {
            'norm': torch.Tensor(trials_in_norm[n_train:n_train+n_valid])
        },
        'labels': torch.Tensor(trials_lbl[n_train:n_train+n_valid]).type(torch.LongTensor)
    }
}

print(f"Train X shape: {trials_ds['train']['samples']['norm'].shape}, Y shape: {trials_ds['train']['labels'].shape}")
print(f"Test X shape: {trials_ds['valid']['samples']['norm'].shape}, Y shape: {trials_ds['valid']['labels'].shape}")

## Detecting control traffic among other traffic

eMBB, URLLC, and mMTC, and include an additional class of samples, denoted as control (ctrl), or “silent”, class to identify the portions of traffic where no application data is being exchanged between registered UEs and gNB


Depending on the UE’s activities, control class traffic might be found in any of the three traffic categories and therefore we consider ctrl as a fourth meta-class that can be used to identify idle users and applications



In [ ]:
all_ctrl = None
columns_maxmin['info'] = {}


# find all data labeled as the control_class
for t in ['train', 'valid']:
    ixs_ctrl = trials_ds[t]['labels'] == ctrl_class
    if torch.any(ixs_ctrl):
        if all_ctrl is None:
            all_ctrl = trials_ds[t]['samples']['norm'][ixs_ctrl]
        else:
            all_ctrl = torch.cat((all_ctrl, trials_ds[t]['samples']['norm'][ixs_ctrl]), dim=0)

if (not (all_ctrl is None)) and (all_ctrl.shape[0] > 0):
    mean_ctrl_sample = torch.mean(all_ctrl, dim=0)
    std_ctrl_sample = torch.std(all_ctrl, dim=0)
    columns_maxmin['info']['mean_ctrl_sample'] = mean_ctrl_sample
    columns_maxmin['info']['std_ctrl_sample'] = std_ctrl_sample
    columns_maxmin['info']['norm_dist'] = {}
    x_axis = np.arange(0, 15, 0.01)
    for cl in [0, 1, 2, 3]:
            all_sample = None
            for t in ['train', 'valid']:
                ixs_ctrl = trials_ds[t]['labels'] == cl
                if torch.any(ixs_ctrl):
                    if all_sample is None:
                        all_sample = trials_ds[t]['samples']['norm'][ixs_ctrl]
                    else:
                        all_sample = torch.cat((all_sample, trials_ds[t]['samples']['norm'][ixs_ctrl]), dim=0)

            norm = np.linalg.norm(all_sample - mean_ctrl_sample, axis=(1, 2))
            columns_maxmin['info']['norm_dist'][cl] = {'mean': np.mean(norm), 'std': np.std(norm)}

ctrl_distrib = normal.pdf(x_axis, columns_maxmin['info']['norm_dist'][3]['mean'], columns_maxmin['info']['norm_dist'][3]['std'])
for cl, traff_name in {0: 'eMBB', 1: 'mMTC', 2: 'URLLC'}.items():
    this_mean, this_std = columns_maxmin['info']['norm_dist'][cl]['mean'], columns_maxmin['info']['norm_dist'][cl]['std']
    this_distrib = normal.pdf(x_axis, this_mean, this_std)
    thr_val = this_mean - (1 * this_std)
    columns_maxmin['info']['norm_dist'][cl]['thr'] = thr_val # save threshold value


### Relabeling

In [ ]:

## Make sure you run this cell only once or run everything from the beginning again


for t in ['train', 'valid']:
    include_ixs = set(range(trials_ds[t]['samples']['norm'].shape[-1]))
    mean_ctrl_sample = columns_maxmin['info']['mean_ctrl_sample']
    obs_excludecols = trials_ds[t]['samples']['norm'][:, :, list(include_ixs)]
    norm = np.linalg.norm(obs_excludecols - mean_ctrl_sample, axis=(1, 2))
    possible_ctrl_ixs = norm < columns_maxmin['info']['norm_dist'][ctrl_class]['mean']
    possible_ctrl_labels = trials_ds[t]['labels'][possible_ctrl_ixs]
    pre_unique_lbls, pre_unique_cnts = np.unique(trials_ds[t]['labels'], return_counts=True)
    print("\ntrain: Initial # samps. per label (before relabeling)\n\t Labels:", pre_unique_lbls[:4], "Count:", pre_unique_cnts[:4])
    unique_labels, unique_counts = np.unique(possible_ctrl_labels, return_counts=True)
    count_relabel = 0
    for ix, isPossibleCTRL in enumerate(possible_ctrl_ixs):
        if isPossibleCTRL and trials_ds[t]['labels'][ix] != ctrl_class:
            #print(ix, self.obs_labels[ix], '-> 3')
            trials_ds[t]['labels'][ix] = ctrl_class
            count_relabel += 1
    print(t, ": Tot. samples relabeled (for every class):", count_relabel)
    pre_unique_lbls, pre_unique_cnts = np.unique(trials_ds[t]['labels'], return_counts=True)
    print(t, ": # samps. per label (after relabeling)\n\t Labels:", pre_unique_lbls[:4], "Count:", pre_unique_cnts[:4])


print("Control classes removed")

## Data Visualization

In [ ]:
embb_mask = trials_ds['valid']['labels'] == 0
mmtc_mask = trials_ds['valid']['labels'] == 1
urllc_mask = trials_ds['valid']['labels'] == 2
ctrl_mask = trials_ds['valid']['labels'] == 3

### Reducing dataset to 2 dimensions visualization using t-SNE method



In many data analysis tasks, we deal with high-dimensional data. For example, each datapoint in our dataset is represented as a vector of size (4, 17), where 4 comes from the window size and 17 represents the number of feature columns. This means that each datapoint can be viewed as a point in a 4x17 dimensional space. But how can we interpret this data in a lower-dimensional space, such as 2D, for better understanding and visualization?

Consider a simple analogy: imagine we have points in 3D space. We can project each point onto a 2D plane within that 3D space, and plot these projections on a 2D surface. This gives us a way to visualize and interpret our 3D data in a much simpler, 2D format.

This idea of reducing high-dimensional data into a more interpretable, lower-dimensional representation is exactly what techniques like t-SNE aim to achieve. t-SNE models pairwise similarities between data points using probability distributions and minimizes the divergence between high-dimensional and low-dimensional representations. t-SNE captures complex non-linear relationships, making it effective for clustering and revealing hidden structures in data.









In [ ]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(trials_ds['valid']['samples']['norm'].reshape((trials_ds['valid']['samples']['norm'].shape[0], -1)))

plt.figure(figsize=(16, 12))
plt.scatter(X_embedded[embb_mask, 0], X_embedded[embb_mask, 1], marker='o', c='red', label='embb')
plt.scatter(X_embedded[mmtc_mask, 0], X_embedded[mmtc_mask, 1], marker='o', c='blue', label='mmtc')
plt.scatter(X_embedded[urllc_mask, 0], X_embedded[urllc_mask, 1], marker='o', c='green', label='urllc')
plt.scatter(X_embedded[ctrl_mask, 0], X_embedded[ctrl_mask, 1], marker='o', c='black', label='cntrl')
plt.legend()
plt.show()


# Model training

So far we have processed our data; labeled it, cleaned it, removed unnessary columns and visualized it. The next step step is to define the maching learning model, train it with the training set and evluate it with the evaluation set.



## CNN Model

### Training configuration

Here we define some of the parameters of the model training process
like the number of iterations we want to refine our ml model defined as `train_config['epochs']` 
or the `window_size` of our model or number of features of our model
which is equal to number of columns of our dataset

In [ ]:

train_config = {}
train_config['Nclass'] = len(np.unique(trials_ds['train']['labels']))
train_config['num_feats'] = trials_ds['train']['samples']['norm'].shape[2]
train_config['logdir'] = "./results"
train_config['window_size'] = window_size
train_config['patience'] = 5                                # Num of epochs to wait before interrupting training with early stopping
train_config['lr'] = 0.001
train_config['lrmin'] = 0.00001
train_config['lrpatience'] = 30
train_config['batch_size'] = 512
train_config['epochs'] = 20
train_config['device'] = torch.device("cpu")


### Batching the data for training

In [ ]:
from torch.utils.data import Dataset

class ORANTracesDataset(Dataset):
    def __init__(self, obs_input, obs_labels):
        self.obs_input, self.obs_labels = obs_input, obs_labels

    def info(self):
        unique_labels, unique_count = np.unique(np.array(self.obs_labels), return_counts=True)
        ds_info = {
            'numfeats': self.obs_input.shape[2],
            'window_size': self.obs_input.shape[1],
            'numsamps': self.obs_input.shape[0],
            'nclasses': len(unique_labels),
            'samples_per_class': unique_count
        }
        return ds_info

    def __len__(self):
        return len(self.obs_input)

    def __getitem__(self, idx):
        obs = self.obs_input[idx, :, :]
        label = self.obs_labels[idx]

        try:
            if self.transform:
                obs = self.transform(obs)
            if self.target_transform:
                label = self.target_transform(label)
        except:
            pass
        return obs, label

ds_train = ORANTracesDataset(trials_ds['train']['samples']['norm'], trials_ds['train']['labels'])
ds_test = ORANTracesDataset(trials_ds['valid']['samples']['norm'], trials_ds['valid']['labels'])
train_dataloader = DataLoader(ds_train, batch_size=train_config['batch_size'], shuffle=True)
test_dataloader = DataLoader(ds_test, batch_size=train_config['batch_size'], shuffle=True)


### Model Definition


#### **Overview**

This machine learning model is a **function estimator** that takes a set of input data points, processes them through several layers of computation, and outputs a prediction. In simple terms, this model learns the relationship between the input data and a set of categories (classes). It does this by processing the input through a series of mathematical operations (layers) and then predicting the class probabilities.


The architecure of the machine learning model is as follows. The model takes an input tensor of shape `(4, 17)`, for a `window_size=4` and the output is a set of 4 values representing the **probabilities** that the input data point belongs to one of the 4 possible classes.


![Model Image](./images/model.jpg)



In [ ]:

class ConvNN(nn.Module):
    def __init__(self, numChannels=1, window_size=4, num_feats=17, classes=4):
        super(ConvNN, self).__init__()
        self.numChannels = numChannels

        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=20,
                               kernel_size=(4, 1))
        self.relu1 = nn.ReLU()

        # pass a random input just to figure our the size of output layer
        rand_x = torch.Tensor(np.random.random((1, 1, window_size, num_feats)))
        output_size = torch.flatten(self.conv1(rand_x)).shape
        
        self.fc1 = nn.Linear(in_features=output_size.numel(), out_features=512)

        # self.fc1 = nn.Linear(in_features=20 * num_feats, out_features=512)

        
        self.relu3 = nn.ReLU()
        # initialize our softmax classifier
        self.fc2 = nn.Linear(in_features=512, out_features=classes)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.reshape \
            ((x.shape[0], self.numChannels, x.shape[1], x.shape[2]))   # CNN 2D expects a [N, Cin, H, W] size of data
        # pass the input through our first set of CONV => RELU =>
        x = self.conv1(x)
        x = self.relu1(x)
        ## flatten the output from the previous layer and pass it
        ## through our only set of FC => RELU layers
        x = torch.flatten(x, 1)       
        x = self.fc1(x)
        x = self.relu3(x)
        # pass the output to our softmax classifier to get our output
        # predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

model = ConvNN(classes=train_config['Nclass'], window_size=train_config['window_size'], 
                                     num_feats=train_config['num_feats']).to(train_config['device'])

In [ ]:
from torchinfo import summary


data_point_dimensions = (trials_in.shape[1], trials_in.shape[2])
print("Our one data point dimensions:", data_point_dimensions)
summary(model, input_size=(1, data_point_dimensions[0], data_point_dimensions[1]), verbose=0, depth=1)  # Adjust input_size based on your model input shape


<!-- #### Loss function

Negative Log-Likelihood Loss is a loss function commonly used for classification tasks. It computes the loss by taking the negative log of the predicted probability assigned to the correct class. This loss is typically used after applying `torch.nn.LogSoftmax()`, which outputs log-probabilities.

### Mathematical Definition:

Given a batch of size $N$ with $C$ classes, let:

- $x_i \in \mathbb{R}^C$ be the log-probabilities for sample $i$.
- $y_i$ be the target class index for sample $i$ (where $y_i \in \{0, 1, \dots, C-1\}$).
- $w_{y_i}$ be an optional weight for class $y_i$.

The NLL loss is computed as:

$$
\text{NLLLoss} = -\frac{1}{N} \sum_{i=1}^{N} w_{y_i} \cdot x_{i, y_i}
$$

where $x_{i, y_i}$ is the log-probability of the correct class for sample $i$.

### Key Points:
- Requires input to be log-probabilities (not raw scores).
- Often used with `LogSoftmax()`, as it outputs log-probabilities.
- Can be weighted to address class imbalance.
- Supports ignoring specific class indices during training.

If no weights are provided, the loss simplifies to:

$$
\text{NLLLoss} = -\frac{1}{N} \sum_{i=1}^{N} x_{i, y_i}
$$
 -->

### Loss function

In machine learning, our model can be seen as a function estimator $ f $, which has parameters that we need to optimize. To do this, we define a **loss function** that quantifies how far off the model’s predictions are from the true values. For every input data point $ x $, the model outputs $ f(x) = [p_1, p_2, p_3, p_4] $, where each $ p_i $ represents the predicted probability for class $ i $. We then proceed to calculate a loss based on this output, then we apply **gradient descent** to adjust the model’s parameters and minimize the loss, ultimately improving the model’s ability to make accurate predictions. We use **Negative Log-Likelihood Loss (NLLLoss)** as the Loss function.




The **Negative Log-Likelihood Loss (NLLLoss)** is calculated as follows: Given the true label is $ y $ for the given input $ x $,  the NLLLoss is computed as:

$$
\text{Loss} = -\log(p_y)
$$

where $ p_y $ is the predicted probability corresponding to the true class $ y $. This loss is then averaged over all input data points in the batch, and the model is optimized to minimize this value, effectively increasing the predicted probability for the correct class during training.


In [ ]:
loss_fn = nn.NLLLoss()

#### Accuracy with the randomly initialized model

In [ ]:
# We have not trained the model so far. The model is only pre populated with initial values
# Now we evaluate the untrained model on the validation dataset.


model.eval()
test_loss, correct = 0, 0
test_dataset_length = len(test_dataloader.dataset)
test_num_batches = len(test_dataloader)

with torch.no_grad():
    for X, y in test_dataloader:
        X = X.to(train_config['device'])
        y = y.to(train_config['device'])
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
test_loss /= test_num_batches
correct /= test_dataset_length
print(f"Validation loss: {test_loss:>8f} , Validation accuracy: {(100 * correct):>0.1f}% \n")


<div style="font-family: Arial, sans-serif; color: #222; line-height: 1.6; max-width: 700px; margin: 20px auto; padding: 20px; border: 1px solid #444; border-radius: 8px; background-color: #eaeaea; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.15);">
  <h2 style="color: #ffffff; background-color: #333333; padding: 10px; border-radius: 4px; margin-bottom: 20px; text-align: center;">Question: The model accuracy</h2>
  
  <p style="font-size: 1.1em; margin-top: 15px; margin-bottom: 20px;">
      Look at the output of the cell above. We are testing the testing the model on the validation set, without having trained the model. Validation accuracy, is the percentage of correct predictions on the validation dataset.      
      As you can see, the validation accuracy is not a number near 0. But we haven't even trained the model. This means model parameters are just randomly selected and not tuned to correctly predict the class given the input. But why do we not get a numer close to zero?
  </p>


  <h3 style="color: #ff9800; margin-bottom: 15px;">Hint</h3>
  <p style="font-size: 1.05em; margin-bottom: 20px;">
      It is true that the model parameters are random. Hence, the output probabilites of the model are random and so the predicted class of the input is also a random number from 1 to 4. However, think of yourself writing a quiz of multi-choice questions with 4 possible answers for every question. How many correct answers would you get if you randomly choose the answers?
  </p>
  
  <h4 style="color: #333333; margin-bottom: 10px; border-bottom: 1px solid #999; padding-bottom: 5px;">Solution:</h4>
  <p style="font-size: 1.05em; margin-bottom: 20px;">
      The answer is almost said in the hint section. You will get a fourth of the answers correctly!
  </p>
</div>



#### Training

<!-- 
1. The Adam optimizer is used for model training.

2. A learning rate scheduler (`ReduceLROnPlateau`) monitors the validation loss and reduces the learning rate when it stops improving. If the learning rate decreases, the model reverts to the last best state to prevent instability.


3. In each training step: we compute model predictions, calculates the loss, perform backpropagation, and update the model parameters using the optimizer.

4. In each validation step, we use `torch.no_grad()` to disable gradient computation (reducing memory usage and computation time), compute validation loss, accuracy, and update the confusion matrix.

5. If the current model achieves a lower validation loss than best_loss, it is saved as the best model. The confusion matrix is also saved.

6. If the model does not improve for `train_config['patience']` epochs, training is stopped early to prevent overfitting.

 -->


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=train_config['lr'])
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=train_config['lrpatience'], min_lr=train_config['lrmin'], 
                              verbose=True)

loss_results = []
best_loss = np.inf
epochs_wo_improvement = 0
times = []
last_lr = np.inf
model_name = f"model.cnn.{train_config['window_size']}.pt"
train_dataset_length = len(train_dataloader.dataset)
train_num_batches = len(train_dataloader)

# Training Epochs
for e in range(train_config['epochs']):
    # Train Step
    model.train()
    start_time = time.time()
    loss_train, correct_train = 0, 0
    for batch, (X, y) in enumerate(train_dataloader):
        # Compute prediction error
        X = X.to(train_config['device'])
        y = y.to(train_config['device'])
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
        correct_train += (pred.argmax(1) == y).type(torch.float).sum().item()
    loss_train /= train_num_batches
    correct_train /= train_dataset_length
    print(f"\nTrainig loss: {loss_train:>7f}, Trainig accuracy: {(100 * correct_train):>0.1f}% \n")

    times.append(time.time() - start_time)

    # Test step
    model.eval()
    test_loss, correct = 0, 0
    conf_matrix = np.zeros((train_config['Nclass'], train_config['Nclass']))
    with torch.no_grad():
        for X, y in test_dataloader:
            X = X.to(train_config['device'])
            y = y.to(train_config['device'])
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            conf_matrix += conf_mat(y.cpu(), pred.argmax(1).cpu(), labels=list(range(train_config['Nclass'])))
    test_loss /= test_num_batches
    correct /= test_dataset_length
    loss, cm = test_loss, conf_matrix
    print(f"Validation loss: {test_loss:>8f} , Validation accuracy: {(100 * correct):>0.1f}% \n")

    # Learning rate adaption
    scheduler.step(loss)
    if scheduler._last_lr[0] < last_lr:    # detect change in lr by scheduler
        if last_lr == np.inf:
            last_lr = scheduler._last_lr[0]
        else:
            print("[lr change detect] -> Reloading from best state, skipping epoch")
            last_lr = scheduler._last_lr[0]
            # Load the last best state for the model if the scheduler has changed.
            # This is to address large instability while training with higher learning rates and avoid
            # continuing from a worse state that might lead to a lower local optima
            model.load_state_dict(torch.load(os.path.join(train_config['logdir'], model_name), map_location=train_config['device'])['model_state_dict'])
            # skip checks and go to next epoch
            continue

    loss_results.append(loss)
    epochs_wo_improvement += 1
    if best_loss > loss:
        pickle.dump(cm, open(os.path.join(train_config['logdir'], f"conf_matrix.cnn.{train_config['window_size']}.last.pkl"), 'wb'))
        epochs_wo_improvement = 0
        best_loss = loss
        torch.save({
             'model_state_dict': model.state_dict(),
             'optimizer_state_dict': optimizer.state_dict(),
             'loss': loss,
        }, os.path.join(train_config['logdir'], model_name))

    if epochs_wo_improvement > train_config['patience']: # early stopping
        print('------------------------------------')
        print('Early termination implemented at epoch:', e+1)
        print('------------------------------------')
        break


# Exercise: Window Size Impact




#### A. Fill the table below by training different models with different window sizes. 

change the window_size in the beginning cells of this notebook. Then run the all cells again from top to bottom. To run all cells, Look at the top bar and select Run->Run All cells. 


|  | window size = 4| window size = 16| window size = 32 |
|----------|----------|----------|----------|
| CNN Training Accuracy| result 1 | result 3 | result 5 |
| CNN Validation Accuracy| result 2 | result 4 | result 6 |


#### B. What is the impact of window size on the performance?
